## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [3]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-11-05 16:15:11--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   183MB/s    in 0.3s    

2024-11-05 16:15:12 (183 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

In [1]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

In [2]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [4]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [8]:
# Função para mapear "sentiment" como chave e "id" como valor do tipo inteiro
def map1(x):
    return (x['sentiment'], int(x['id']))

# Dados de exemplo
data = [
    {'sentiment': 'positive', 'id': '101'},
    {'sentiment': 'negative', 'id': '102'},
    {'sentiment': 'negative', 'id': '105'},
    {'sentiment': 'neutral', 'id': '103'},
    {'sentiment': 'negative', 'id': '107'}
]

# Mapeia os dados
mapped_data = list(map(map1, data))

# Filtra apenas os elementos com "sentiment" negativo e soma os IDs
soma_ids_negativos = sum(id_value for sentiment, id_value in mapped_data if sentiment == 'negative')

print(f"O valor da soma de todos os IDs dos filmes classificados como negativos é: {soma_ids_negativos}")


O valor da soma de todos os IDs dos filmes classificados como negativos é: 314


## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [11]:
def reduceByKey1(x, y):
    # x e y são os IDs que precisam ser somados
    return x + y

data = [('positive', 1), ('negative', 2), ('positive', 3), ('neutral', 1), ('negative', 1)]

from collections import defaultdict

# Inicializa um dicionário para armazenar a soma dos IDs por sentimento
sentiment_sums = defaultdict(int)

# Percorre os dados e acumula os IDs
for sentiment, id in data:
    sentiment_sums[sentiment] = reduceByKey1(sentiment_sums[sentiment], id)

# Exibe o resultado
print(dict(sentiment_sums))  # {'positive': 4, 'negative': 3, 'neutral': 1}



{'positive': 4, 'negative': 3, 'neutral': 1}


## Aplicação do map/reduce e visualização do resultado

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Inicializa a Spark Session
spark = SparkSession.builder \
    .appName("MapReduce Example") \
    .getOrCreate()

# Exemplo de dados
data = [
    ('positive', 1),
    ('negative', 2),
    ('positive', 3),
    ('neutral', 1),
    ('negative', 1)
]

# Criação do DataFrame
df = spark.createDataFrame(data, ['sentiment', 'id'])

# Aplicando MapReduce: agrupando por 'sentiment' e somando os IDs
result = df.groupBy('sentiment').agg(F.sum('id').alias('total_id'))

# Coletando os resultados
collected_results = result.collect()

# Exibindo os resultados
for row in collected_results:
    print(row)



Row(sentiment='positive', total_id=4)
Row(sentiment='neutral', total_id=1)
Row(sentiment='negative', total_id=3)


# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e uma tupla com a soma das palavras de cada texto como valor.

In [14]:
def map2(x):
    # x é uma tupla no formato (sentiment, text)
    sentiment = x[0]  # O sentimento é o primeiro elemento
    text = x[1]      # O texto é o segundo elemento

    # Contamos o número de palavras no texto
    word_count = len(text.split())  # Divide o texto em palavras e conta

    # Retorna uma tupla com o sentimento e a contagem de palavras
    return (sentiment, (word_count,))

# Exemplo de dados
data = [
    ('positive', 'I love this product!'),
    ('negative', 'This is the worst experience ever.'),
    ('neutral', 'It was okay, nothing special.'),
    ('positive', 'Absolutely fantastic service!')
]

# Aplicando a função map2 a cada elemento dos dados
mapped_results = [map2(item) for item in data]

# Exibindo os resultados mapeados
print(mapped_results)



[('positive', (4,)), ('negative', (6,)), ('neutral', (5,)), ('positive', (3,))]


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [23]:
def reduceByKey2(x, y):
    # Conta o número de palavras no texto x
    palavras_x = len(x.split())

    # Conta o número de palavras no texto y
    palavras_y = len(y.split())

    # Retorna a soma das palavras de ambos os textos
    return palavras_x + palavras_y

# Dados de exemplo, onde cada tupla é ('sentimento', 'texto')
dados = [
    ("positivo", "Eu amo aprender novas coisas."),
    ("negativo", "Eu detesto quando as coisas não dão certo."),
    ("positivo", "A vida é maravilhosa."),
    ("negativo", "Estou muito frustrado com o que aconteceu."),
]

# Agora vamos agrupar os dados por sentimento e aplicar o reduce
from itertools import groupby
from functools import reduce

# Ordena os dados pela chave (sentimento)
dados.sort(key=lambda x: x[0])  # Necessário para usar groupby corretamente

# Agrupar os dados por sentimento
resultados = {}
for chave, grupo in groupby(dados, key=lambda x: x[0]):
    textos = [item[1] for item in grupo]  # Coleta os textos associados ao sentimento
    # Aplica a função de redução para contar o número total de palavras
    total_palavras = reduce(reduceByKey2, textos)
    resultados[chave] = total_palavras

# Exibe o resultado
print(resultados)

dados = [
    ("positivo", "Eu amo aprender novas coisas."),
    ("negativo", "Eu detesto quando as coisas não dão certo."),
    ("positivo", "A vida é maravilhosa."),
    ("negativo", "Estou muito frustrado com o que aconteceu."),
]


{'negativo': 15, 'positivo': 9}


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Iniciar a sessão do Spark
spark = SparkSession.builder.appName("MapReduceTextExample").getOrCreate()

dados = [
    ("positivo", "Eu amo aprender novas coisas."),
    ("negativo", "Eu detesto quando as coisas não dão certo."),
    ("positivo", "A vida é maravilhosa."),
    ("negativo", "Estou muito frustrado com o que aconteceu."),
]

# Criar DataFrame do Spark
df = spark.createDataFrame(dados, ["sentiment", "text"])

# Exibir o DataFrame
df.show(truncate=False)

+---------+------------------------------------------+
|sentiment|text                                      |
+---------+------------------------------------------+
|positivo |Eu amo aprender novas coisas.             |
|negativo |Eu detesto quando as coisas não dão certo.|
|positivo |A vida é maravilhosa.                     |
|negativo |Estou muito frustrado com o que aconteceu.|
+---------+------------------------------------------+

